In [3]:
from relapse_prediction import constants

import pandas as pd
import numpy as np
from tqdm.auto import tqdm


In [19]:
imaging = "OEF"

In [20]:
d_thresholds =  {
    "L3R": 0.36,
    "L3R - (L1 + L3)": 0.33
}

In [21]:
d_confusion_matrix = {
    "L3R": {"tp": 0, "fp": 0, "tn": 0, "fn": 0},
    "L3R - (L1 + L3)": {"tp": 0, "fp": 0, "tn": 0, "fn": 0}
}


In [22]:
for patient in tqdm(constants.list_patients):

    path_features = constants.dir_features / patient / fr"{patient}_{imaging}_features.parquet"
    df_features = pd.read_parquet(path_features, engine="pyarrow")
    df_features = df_features[["x", "y", "z", "mean_5x5x5"]]

    path_labels = constants.dir_labels / fr"{patient}_labels.parquet"
    df_labels = pd.read_parquet(path_labels, engine="pyarrow")
    df_labels = df_labels[["x", "y", "z", "L3R", "L3R - (L1 + L3)"]]

    df_data = df_labels.merge(df_features, on=["x", "y", "z"], how="left")

    for label in ["L3R", "L3R - (L1 + L3)"]:
        df_data[f"predicted_{label}"] = (df_data['mean_5x5x5'] >= d_thresholds[label]).astype(int)
        
        tp = ((df_data[f"predicted_{label}"] == 1) & (df_data[label] == 1)).sum()
        fp = ((df_data[f"predicted_{label}"] == 1) & (df_data[label] == 0)).sum()
        tn = ((df_data[f"predicted_{label}"] == 0) & (df_data[label] == 0)).sum()
        fn = ((df_data[f"predicted_{label}"] == 0) & (df_data[label] == 1)).sum()

        # Aggregate the values
        d_confusion_matrix[label]["tp"] += tp
        d_confusion_matrix[label]["fp"] += fp
        d_confusion_matrix[label]["tn"] += tn
        d_confusion_matrix[label]["fn"] += fn


  0%|          | 0/104 [00:00<?, ?it/s]

In [23]:
for label in ["L3R", "L3R - (L1 + L3)"]:

    recall = d_confusion_matrix[label]["tp"] / (d_confusion_matrix[label]["tp"] + d_confusion_matrix[label]["fn"])
    specificity = d_confusion_matrix[label]["tn"] / (d_confusion_matrix[label]["tn"] + d_confusion_matrix[label]["fp"])

    print(f"Total Recall for label={label} : {recall * 100:.2f} %")
    print(f"Total Specificity for label={label} : {specificity* 100:.2f} %")
    

Total Recall for label=L3R : 51.85 %
Total Specificity for label=L3R : 67.51 %
Total Recall for label=L3R - (L1 + L3) : 59.27 %
Total Specificity for label=L3R - (L1 + L3) : 59.73 %


In [3]:
d = {"a": 1, "b": 2}
d2 = {"b": 3}

In [4]:
{**d, **d2}

{'a': 1, 'b': 3}